In [1]:
from g2opy import g2opy as g2o
import numpy as np

# Landmark sensing noise
noise = 0.02

Create all g2o elements: optimizer, solver and algorithm.

In [2]:
optimizer = g2o.SparseOptimizer()
solver = g2o.BlockSolverX(g2o.LinearSolverDenseX())
algorithm = g2o.OptimizationAlgorithmLevenberg(solver)
optimizer.set_algorithm(algorithm)

Create 1000 landmarks (2D points)

In [3]:
true_points = np.hstack([
    np.random.random((1000, 1)) * 3 - 1.5,
    np.random.random((1000, 1)) - 0.5])

print(true_points)

[[ 0.45578976  0.33228071]
 [-0.55387114  0.0107115 ]
 [-0.88197108 -0.05861287]
 ...
 [-1.0049818  -0.34711565]
 [-0.7501989  -0.24020415]
 [ 0.61243225 -0.35520149]]


Create two robot estimated positions:
 - Vertex 0 is at (0, 0) with orientation 0 deg
 - Vertex 1 is at (1, 0) with orientation 0 deg

Add odometric edge (EdgeSE2) between vertices 0 and 1

In [4]:

# Add two vertices at two robot positions
for i in range(2):
    t = np.array([0, i])
    odometry = g2o.SE2(i, 0, 0)
    vc = g2o.VertexSE2()
    vc.set_id(i)
    vc.set_estimate(odometry)
    if i == 0:
        vc.set_fixed(True)
    optimizer.add_vertex(vc)
    
    
# Add edges between these robot positions
edge = g2o.EdgeSE2()
edge.set_vertex(0, optimizer.vertex(0))
edge.set_vertex(1, optimizer.vertex(1))
edge.set_measurement(g2o.SE2(i, 0, 0))
edge.set_information(np.array(
    [[1, 0, 0], 
     [0, 1, 0], 
     [0, 0, 1]]))
optimizer.add_edge(edge)

True

Prepare inverse transformation for relative landmark measurements

In [5]:
trans0 = optimizer.vertex(0).estimate().inverse()
trans1 = optimizer.vertex(1).estimate().inverse()

print(trans0.translation(), trans0.rotation().angle())
print(trans1.translation(), trans1.rotation().angle())

[ 0. -0.] 0.0
[-1. -0.] 0.0


Create landmark observations as VertexPointXY and their measurements from the robot poses as EdgeSE2PointXY.
In this example, all robot poses see all landmarks. In a real scenario, we'd add all landmarks but only add the edges that we've measured.

In [6]:
# set up point matches
for i in range(len(true_points)):
    # calculate the relative 3d position of the point
    pt0 = trans0 * true_points[i]
    pt1 = trans1 * true_points[i]

    # add noise
    pt0 += np.random.randn(2) * noise
    pt1 += np.random.randn(2) * noise

    landmark = g2o.VertexPointXY()
    landmark.set_id(3 + i)
    landmark.set_estimate(true_points[i])
    optimizer.add_vertex(landmark)
    

    edge = g2o.EdgeSE2PointXY()
    edge.set_vertex(0, optimizer.vertex(0))
    edge.set_vertex(1, optimizer.vertex(2 + i))
    edge.set_measurement(pt0)
    edge.set_information(np.identity(2))
    
    edge = g2o.EdgeSE2PointXY()
    edge.set_vertex(0, optimizer.vertex(1))
    edge.set_vertex(1, optimizer.vertex(2 + i))
    edge.set_measurement(pt1)
    edge.set_information(np.identity(2))

    optimizer.add_edge(edge)

In [7]:
# move second robot pose off of its true position
vc = optimizer.vertex(1)
cam = g2o.SE2(0.2, 0, 0)
vc.set_estimate(cam)

In [8]:
cam.translation()

array([0.2, 0. ])

In [9]:
optimizer.initialize_optimization()
optimizer.compute_active_errors()
print('Initial chi2 =', optimizer.chi2())

Initial chi2 = 2309.6300256203826


In [10]:
optimizer.set_verbose(True)
optimizer.optimize(5)

print('\nSecond vertex should be near [1, 0, 0]')
print('before optimization:', cam.translation())
print('after  optimization:', optimizer.vertex(1).estimate().translation())


Second vertex should be near [1, 0, 0]
before optimization: [0.2 0. ]
after  optimization: [9.99912540e-01 5.97132866e-05]


iteration= 0	 chi2= 0.239075	 time= 0.671896	 cumTime= 0.671896	 edges= 1000	 schur= 0	 lambda= 0.003333	 levenbergIter= 1
iteration= 1	 chi2= 0.000012	 time= 0.635748	 cumTime= 1.30764	 edges= 1000	 schur= 0	 lambda= 0.001111	 levenbergIter= 1
iteration= 2	 chi2= 0.000000	 time= 0.623196	 cumTime= 1.93084	 edges= 1000	 schur= 0	 lambda= 0.000741	 levenbergIter= 1
iteration= 3	 chi2= 0.000000	 time= 0.614513	 cumTime= 2.54535	 edges= 1000	 schur= 0	 lambda= 0.000494	 levenbergIter= 1
iteration= 4	 chi2= 0.000000	 time= 0.627083	 cumTime= 3.17244	 edges= 1000	 schur= 0	 lambda= 0.000329	 levenbergIter= 1
